In [1]:
import numpy as np
from scipy.stats import rankdata
import pandas as pd

In [2]:
alternatif = ['Wiji','Sugiyem','Dul','Kuswara','Tugiyah']
kriteria = ['penghasilan','tanggungan','atap','dinding','komunikasi']
data = np.array([
            [4,4,3,5,3],
            [3,2,3,5,2],
            [4.5,5,2,2,3],
            [3.5,3,3,5,3],
            [3,2,3,5,2]
                ],dtype=np.float128)


# Normalize the decision matrix
normalized_data = data / np.sqrt(np.sum(data**2,axis=0))
# normalized_data

print("DATA")
pd.DataFrame(data,columns=kriteria,index=alternatif)


DATA


,penghasilan,tanggungan,atap,dinding,komunikasi
Wiji,4.0,4.0,3.0,5.0,3.0
Sugiyem,3.0,2.0,3.0,5.0,2.0
Dul,4.5,5.0,2.0,2.0,3.0
Kuswara,3.5,3.0,3.0,5.0,3.0
Tugiyah,3.0,2.0,3.0,5.0,2.0


In [3]:
weights = np.array([5,4,4,5,3])
# weights = np.array([1,4/5,4/5,1,3/5])
print('PEMBOBOTAN')

pd.DataFrame([weights],columns=kriteria,index=['bobot'])

PEMBOBOTAN


,penghasilan,tanggungan,atap,dinding,komunikasi
bobot,5,4,4,5,3


In [4]:
print('Nomalized')
normalized_data


Nomalized


array([[0.49051147, 0.52522573, 0.47434165, 0.49029034, 0.50709255],
       [0.3678836 , 0.26261287, 0.47434165, 0.49029034, 0.3380617 ],
       [0.55182541, 0.65653216, 0.31622777, 0.19611614, 0.50709255],
       [0.42919754, 0.3939193 , 0.47434165, 0.49029034, 0.50709255],
       [0.3678836 , 0.26261287, 0.47434165, 0.49029034, 0.3380617 ]],
      dtype=float128)

In [5]:
print('Weighted')
weighted = normalized_data * weights
weighted

Weighted


array([[2.45255736, 2.10090293, 1.8973666 , 2.45145169, 1.52127766],
       [1.83941802, 1.05045146, 1.8973666 , 2.45145169, 1.01418511],
       [2.75912703, 2.62612866, 1.26491106, 0.98058068, 1.52127766],
       [2.14598769, 1.57567719, 1.8973666 , 2.45145169, 1.52127766],
       [1.83941802, 1.05045146, 1.8973666 , 2.45145169, 1.01418511]],
      dtype=float128)

In [6]:
#True jika  benefit dan False jika Cost
ben_or_cost = np.array([False,True,True,True,True])


In [7]:


maximizing = np.where(ben_or_cost,
                      np.max(weighted,axis=0),
                      np.min(weighted,axis=0))
maximizing

array([1.83941802, 2.62612866, 1.8973666 , 2.45145169, 1.52127766],
      dtype=float128)

In [8]:
minimizing = np.where(ben_or_cost,
                      np.min(weighted,axis=0),
                      np.max(weighted,axis=0))
minimizing

array([2.75912703, 1.05045146, 1.26491106, 0.98058068, 1.01418511],
      dtype=float128)

In [9]:
solusi_ideal_positif = np.sqrt(np.sum((maximizing-weighted)**2, 
                                   axis=1))


solusi_ideal_positif

array([0.8073425 , 1.65526478, 1.84643608, 1.09427293, 1.65526478],
      dtype=float128)

In [10]:
solusi_ideal_negatif = np.sqrt(np.sum((weighted - minimizing)**2, 
                                   axis=1))

solusi_ideal_negatif

array([2.00450434, 1.84643608, 1.65526478, 1.86343938, 1.84643608],
      dtype=float128)

In [11]:
relative_closeness = solusi_ideal_negatif / (solusi_ideal_negatif+solusi_ideal_positif)

# Rank the alternatives
# ranked_alternatives = np.argsort(relative_closeness)
# ranked_alternatives.reshape([5,1] )

print('RELATIVE_CLOSENESS')
relative_closeness

# relative_closeness,ranked_alternatives.reshape([data.shape[0],1])


RELATIVE_CLOSENESS


array([0.71287821, 0.52729692, 0.47270308, 0.63002726, 0.52729692],
      dtype=float128)

In [12]:
print('Ranking')
ranked_alternatives = rankdata(relative_closeness,method='ordinal')
pd.DataFrame( ranked_alternatives.reshape([data.shape[0],1]),index=alternatif)

Ranking


,0
Wiji,5
Sugiyem,2
Dul,1
Kuswara,4
Tugiyah,3


In [13]:

print("Mencoba menggunakan skcriteria TOPSIS, untuk membandingkan, mungkin saya salah ketika menggunakan operasi matriks manual numpy, karena hasilnya beda")
print("\nwalau menggunakan scaler, hasilnya tetap sama, berbeda dengan perhitungan manual sebelumnya")
import skcriteria as skc
from skcriteria.madm import simple
from skcriteria.madm import similarity
from skcriteria.pipeline import mkpipe
from skcriteria.preprocessing import invert_objectives,scalers

minmax = np.where(ben_or_cost,max,min)
dm = skc.mkdm(data,minmax,weights,
              alternatives=alternatif,
              criteria=kriteria)


pipe = mkpipe(
    invert_objectives.NegateMinimize(),
    # scalers.VectorScaler(target='matrix'),
    # scalers.SumScaler(target='weights'),
    similarity.TOPSIS()
)
dm = pipe.evaluate(dm)
dm

Mencoba menggunakan skcriteria TOPSIS, untuk membandingkan, mungkin saya salah ketika menggunakan operasi matriks manual numpy, karena hasilnya beda

walau menggunakan scaler, hasilnya tetap sama, berbeda dengan perhitungan manual sebelumnya


Alternatives,Wiji,Sugiyem,Dul,Kuswara,Tugiyah
Rank,1,3,4,2,3
